# Création d'un dictionnaire qui contient un tableau pour chaque ligne de métro

## Chaque tableau affiche les stations de la ligne, leur fréquentaiton annuelle, leur fréquentation hebdomadaire moyenne sur la période 24/07/2022-11/08/2022 (période des JO mais en 2022)

In [135]:
%pip install openpyxl
import pandas as pd
import numpy as np
import re
import difflib
from IPython.utils import io
from IPython.display import display


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Importer le notebook Nettoyage données affluence par jour + nombre de validation par jour par station

In [136]:
try:
    with io.capture_output():
        %run -i "Nettoyage données affluence par jour + nombre de validation par jour par station.ipynb"
except Exception as e:
    display(e)

Importer les données disponibles sur le site https://data.ratp.fr .
Ce jeu de données présente le trafic des entrants directs sur le réseau ferré RATP en 2021

In [137]:
tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

print("Tableau importé : trafic des entrants directs sur le réseau ferré RATP en 2021")
display(tableau_fréquentation)


Tableau importé : trafic des entrants directs sur le réseau ferré RATP en 2021


,Rang,Réseau,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,Ville,Arrondissement pour Paris
0,7,Métro,REPUBLIQUE,11079708,3,5,8,9.0,11.0,Paris,11.0
1,9,Métro,LA DEFENSE,9256802,1,NaN,NaN,NaN,NaN,Puteaux,NaN
2,13,Métro,HOTEL DE VILLE,7251729,1,11,NaN,NaN,NaN,Paris,4.0
3,30,Métro,VILLEJUIF-LOUIS ARAGON,5218070,7,NaN,NaN,NaN,NaN,Villejuif,NaN
4,40,Métro,PALAIS-ROYAL,4822599,1,7,NaN,NaN,NaN,Paris,1.0
...,...,...,...,...,...,...,...,...,...,...,...
366,33,RER,NOGENT-SUR-MARNE,2358276,A,NaN,NaN,NaN,NaN,Nogent-sur-Marne,NaN
367,43,RER,BAGNEUX,1742783,B,NaN,NaN,NaN,NaN,Cachan,NaN
368,44,RER,LE PARC DE SAINT-MAUR,1617366,A,NaN,NaN,NaN,NaN,Saint-Maur des Fossés,NaN
369,54,RER,PALAISEAU,694838,B,NaN,NaN,NaN,NaN,Palaiseau,NaN


Nettoyage du tableau

In [138]:
tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
tableau_fréquentation = tableau_fréquentation.drop_duplicates(subset='Station')

colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

display(tableau_fréquentation)

,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5
0,7,REPUBLIQUE,11079708,3,5,8,9.0,11.0
1,9,LA DEFENSE,9256802,1,nan,nan,nan,nan
2,13,HOTEL DE VILLE,7251729,1,11,nan,nan,nan
3,30,VILLEJUIF-LOUIS ARAGON,5218070,7,nan,nan,nan,nan
4,40,PALAIS-ROYAL,4822599,1,7,nan,nan,nan
...,...,...,...,...,...,...,...,...
357,286,PICPUS,931602,6,nan,nan,nan,nan
358,287,TEMPLE,891858,3,nan,nan,nan,nan
359,289,QUAI DE LA RAPEE,798728,5,nan,nan,nan,nan
360,299,DANUBE,400157,7bis,nan,nan,nan,nan


In [139]:
# Ajouter la colonne 'nb_correspondance'
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].apply(lambda row: sum(pd.notna(value) and value != 'nan' for value in row), axis=1)

# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']

display(tableau_fréquentation)

,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nb_correspondance
0,7,REPUBLIQUE,2215941.6,3,5,8,9.0,11.0,5
1,9,LA DEFENSE,9256802.0,1,nan,nan,nan,nan,1
2,13,HOTEL DE VILLE,3625864.5,1,11,nan,nan,nan,2
3,30,VILLEJUIF-LOUIS ARAGON,5218070.0,7,nan,nan,nan,nan,1
4,40,PALAIS-ROYAL,2411299.5,1,7,nan,nan,nan,2
...,...,...,...,...,...,...,...,...,...
357,286,PICPUS,931602.0,6,nan,nan,nan,nan,1
358,287,TEMPLE,891858.0,3,nan,nan,nan,nan,1
359,289,QUAI DE LA RAPEE,798728.0,5,nan,nan,nan,nan,1
360,299,DANUBE,400157.0,7bis,nan,nan,nan,nan,1


Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne 

In [140]:
numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in str(x.values), axis=1)]
    
    # Supprimer les doublons dans la colonne 'Station'
    filtered_rows = filtered_rows.drop_duplicates(subset='Station')
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)
    tableau = tableau.drop_duplicates(subset='Station')
    tableaux[tableau_nom] = tableau.drop_duplicates(subset='Station')  # Ajoutez cette ligne pour supprimer les doublons de la colonne 'Station'


In [141]:

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Part du trafic de la station par rapport à toute la ligne'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

In [142]:
#compléter avec les valeurs manquantes

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.4)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



In [143]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

In [144]:
for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les doublons de stations
    current_table = current_table.drop_duplicates(subset='Station')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table


In [145]:
for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table

In [146]:
for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table


In [147]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic',
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne',
    'fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]

# Vérifier que rien n'a été oublié
print("Colonnes disponibles :")
print(tableau.columns)
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    
    display(tableau)
    print("\n" + "=" * 50 + "\n")

Colonnes disponibles :
Index(['Rang', 'Station', 'Trafic', 'Correspondance_1', 'Correspondance_2',
       'Correspondance_3', 'Correspondance_4', 'Correspondance_5',
       'nb_correspondance', 'fréquentation_totale_ligne',
       'Part du trafic de la station par rapport à toute la ligne',
       'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
       '00:00-01:00', '01:00-02:00', '02:00-03:00', '03:00-04:00',
       '04:00-05:00', '05:00-06:00', '06:00-07:00', '07:00-08:00',
       '08:00-09:00', '09:00-10:00', '10:00-11:00', '11:00-12:00',
       '12:00-13:00', '13:00-14:00', '14:00-15:00', '15:00-16:00',
       '16:00-17:00', '17:00-18:00', '18:00-19:00', '19:00-20:00',
       '20:00-21:00', '21:00-22:00', '22:00-23:00', '23:00-00:00',
       'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
       'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction',
       'nombre_validations_totales_pour_la_ligne'],
      dtype='object')
Tableau: ligne numéro[1]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,republique,2215941.6,0.007421,3676.321623,735.264325,367.632162,5,3,5,8,...,38.42,42.22,48.75,47.65,40.31,26.59,22.13,22.07,20.89,7
1,la defense,9256802.0,0.031002,5635.386074,5635.386074,2817.693037,1,1,nan,nan,...,68.93,79.89,103.14,108.83,84.18,53.23,26.30,14.93,14.40,9
2,hotel de ville,3625864.5,0.012143,2411.033293,1205.516646,602.758323,2,1,11,nan,...,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49,13
3,palais-royal,2411299.5,0.008076,1657.541667,828.770833,414.385417,2,1,7,nan,...,40.91,55.84,70.75,67.08,42.63,27.29,21.24,20.44,20.58,40
4,basilique de saint-denis,3991395.0,0.013367,348.549592,348.549592,174.274796,1,13,nan,nan,...,34.14,36.42,39.92,33.92,25.82,14.12,8.43,6.61,4.61,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,jussieu,1444821.0,0.004839,1019.405340,509.702670,254.851335,2,7,10,nan,...,38.64,47.61,62.35,61.45,41.98,26.16,18.39,17.92,17.77,114
124,mairie d'issy,2874138.0,0.009626,979.186173,979.186173,489.593087,1,12,nan,nan,...,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20,115
125,pernety,2173567.0,0.007279,692.335762,692.335762,346.167881,1,13,nan,nan,...,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96,162
126,notre-dame-de-lorette,1893828.0,0.006343,639.111922,639.111922,319.555961,1,12,nan,nan,...,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33,188




Tableau: ligne numéro[2]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,concorde,1.133740e+06,0.009798,1252.275152,417.425051,208.712525,3,1,8,12,...,40.62,54.21,66.58,69.70,54.94,27.15,19.30,18.36,17.88,86
1,pasteur,1.513143e+06,0.013077,1109.536689,554.768344,277.384172,2,6,12,nan,...,35.89,38.08,43.50,44.12,35.49,20.42,12.83,11.10,10.56,105
2,victor hugo,2.801041e+06,0.024207,830.791135,830.791135,415.395568,1,2,nan,nan,...,39.10,43.64,48.86,48.08,43.57,22.99,12.69,12.26,12.33,123
3,alexandre dumas,2.476608e+06,0.021403,776.476449,776.476449,388.238225,1,2,nan,nan,...,36.13,39.34,43.01,39.28,33.30,22.87,14.42,12.51,10.12,137
4,abbesses,1.513884e+06,0.013083,386.886573,386.886573,193.443286,1,12,nan,nan,...,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35,233
5,trinite-d'estienne d'orves,1.051982e+06,0.009091,531.785099,531.785099,265.892549,1,12,nan,nan,...,38.18,44.55,51.19,54.76,49.49,33.64,26.85,21.44,16.85,280
6,saint-georges,7.515550e+05,0.006495,244.403647,244.403647,122.201824,1,12,nan,nan,...,37.65,38.83,47.79,42.88,34.41,21.11,16.17,26.32,17.00,291
7,assemblee nationale,6.115120e+05,0.005285,233.615898,233.615898,116.807949,1,12,nan,nan,...,35.53,47.38,65.30,64.65,41.95,25.79,16.80,15.93,18.23,296
8,saint-lazare,6.625677e+06,0.057259,18616.056763,3723.211353,1861.605676,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
10,nation,1.512699e+06,0.013073,1605.467995,401.366999,200.683499,4,1,2,6,...,73.30,78.47,90.17,88.67,79.27,49.96,33.56,27.09,22.85,20




Tableau: ligne numéro[3]


...

,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,concorde,1.133740e+06,0.018686,1252.275152,417.425051,208.712525,3,1,8,12,...,40.62,54.21,66.58,69.70,54.94,27.15,19.30,18.36,17.88,86
1,pasteur,1.513143e+06,0.024939,1109.536689,554.768344,277.384172,2,6,12,nan,...,35.89,38.08,43.50,44.12,35.49,20.42,12.83,11.10,10.56,105
2,abbesses,1.513884e+06,0.024951,386.886573,386.886573,193.443286,1,12,nan,nan,...,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35,233
3,trinite-d'estienne d'orves,1.051982e+06,0.017338,531.785099,531.785099,265.892549,1,12,nan,nan,...,38.18,44.55,51.19,54.76,49.49,33.64,26.85,21.44,16.85,280
4,saint-georges,7.515550e+05,0.012387,244.403647,244.403647,122.201824,1,12,nan,nan,...,37.65,38.83,47.79,42.88,34.41,21.11,16.17,26.32,17.00,291
5,assemblee nationale,6.115120e+05,0.010079,233.615898,233.615898,116.807949,1,12,nan,nan,...,35.53,47.38,65.30,64.65,41.95,25.79,16.80,15.93,18.23,296
6,saint-lazare,6.625677e+06,0.109201,18616.056763,3723.211353,1861.605676,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
8,marcadet-poissonniers,1.991002e+06,0.032815,1335.092784,667.546392,333.773196,2,4,12,nan,...,37.81,39.17,43.07,42.94,37.58,27.00,16.68,10.82,7.56,55
9,convention,3.734750e+06,0.061554,1207.160606,1207.160606,603.580303,1,12,nan,nan,...,38.31,38.63,37.75,38.46,33.51,20.97,13.06,9.80,7.74,67
10,pigalle,1.750916e+06,0.028858,1056.782899,528.391449,264.195725,2,2,12,nan,...,34.97,37.66,39.98,40.12,36.95,26.15,21.85,28.69,26.75,77




Tableau: ligne numéro[13]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,basilique de saint-denis,3991395.0,0.048066,348.549592,348.549592,174.274796,1,13,nan,nan,...,34.14,36.42,39.92,33.92,25.82,14.12,8.43,6.61,4.61,54
1,plaisance,3521753.0,0.042411,1134.947464,1134.947464,567.473732,1,13,nan,nan,...,34.61,37.11,39.72,38.27,32.66,22.15,12.53,8.94,7.01,76
2,malakoff-plateau de vanves,2242320.0,0.027003,773.738210,773.738210,386.869105,1,13,nan,nan,...,35.59,34.75,39.62,41.65,31.21,18.21,9.74,7.87,5.92,155
3,guy moquet,2205313.0,0.026557,894.240752,894.240752,447.120376,1,13,nan,nan,...,34.54,35.67,37.81,36.78,33.77,20.83,12.24,9.86,8.41,158
4,champs-elysees-clemenceau,954502.5,0.011495,636.167476,318.083738,159.041869,2,1,13,nan,...,34.39,47.75,62.33,67.18,52.39,31.77,25.55,24.32,23.10,187
5,les agnettes,1890356.0,0.022765,610.874772,610.874772,305.437386,1,13,nan,nan,...,32.00,33.13,34.57,31.37,25.58,17.10,10.05,7.16,5.00,189
6,la fourche,1829271.0,0.022029,550.641119,550.641119,275.320560,1,13,nan,nan,...,33.81,36.30,40.81,40.02,37.61,24.24,15.98,12.67,11.38,198
7,varenne,782697.0,0.009426,270.470695,270.470695,135.235347,1,13,nan,nan,...,39.96,55.50,69.74,68.24,34.46,17.61,11.87,10.16,16.57,290
8,saint-lazare,6625676.8,0.079790,18616.056763,3723.211353,1861.605676,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
10,chatillon-montrouge,5034012.0,0.060622,1543.318237,1543.318237,771.659118,1,13,nan,nan,...,32.62,34.96,41.77,41.56,29.17,16.87,9.27,6.52,5.61,35




Tableau: ligne numéro[14]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,saint-lazare,6625676.8,0.119400,18616.056763,3723.211353,1861.605676,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
2,mairie de saint-ouen,2415405.0,0.043528,846.124381,423.062191,211.531095,2,13,14,nan,...,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66,39
3,pont cardinet,4168538.0,0.075121,1497.119048,1497.119048,748.559524,1,14,nan,nan,...,35.56,37.63,43.22,45.32,37.99,23.12,14.08,7.68,6.32,50
4,pyramides,1981349.0,0.035706,1287.978261,643.989130,321.994565,2,7,14,nan,...,40.24,48.26,63.20,67.67,51.92,33.49,24.30,22.48,18.01,56
5,clichy saint-ouen,3420852.0,0.061647,1180.762691,1180.762691,590.381346,1,14,nan,nan,...,34.42,42.10,54.71,54.96,36.11,20.36,12.13,6.13,4.06,84
6,bibliotheque,11104474.0,0.200112,2272.627107,2272.627107,1136.313554,1,14,nan,nan,...,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76,6
7,chatelet,1670158.8,0.030098,5877.878321,1175.575664,587.787832,5,1,4,7,...,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60,10
8,olympiades,5214595.0,0.093971,1895.432969,1895.432969,947.716484,1,14,nan,nan,...,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47,31
9,gare de lyon,9546825.0,0.172042,5298.853444,1766.284481,883.142241,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
11,bercy,1942106.0,0.034998,1407.759105,703.879553,351.939776,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61


In [148]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic', 
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne','fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]

# Vérifier que rien n'a été oublié
print("Colonnes disponibles :")


Colonnes disponibles :


In [149]:
# Nouveaux noms des colonnes
nouveaux_noms_colonnes = {
    'Station': 'Station',
    'Trafic': 'Trafic annuel dans la station',
    'Part du trafic de la station par rapport à toute la ligne': 'Part de la station dans le trafic annuel de la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station': 'Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne': 'Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction': 'Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO',
    'Correspondance_1': 'Correspondance 1',
    'Correspondance_2': 'Correspondance 2',
    'Correspondance_3': 'Correspondance 3',
    'Correspondance_4': 'Correspondance 4',
    'Correspondance_5': 'Correspondance 5',
    'nb_correspondance': 'Nombre de correspondances de la station',
    'nombre_validations_totales_pour_la_ligne': 'Nombre de validations annuelles sur la ligne',
    'fréquentation_totale_ligne': 'Fréquentation totale de la ligne',
    '00:00-01:00': '00:00-01:00',
    '01:00-02:00': '01:00-02:00',
    '02:00-03:00': '02:00-03:00',
    '03:00-04:00': '03:00-04:00',
    '04:00-05:00': '04:00-05:00',
    '05:00-06:00': '05:00-06:00',
    '06:00-07:00': '06:00-07:00',
    '07:00-08:00': '07:00-08:00',
    '08:00-09:00': '08:00-09:00',
    '09:00-10:00': '09:00-10:00',
    '10:00-11:00': '10:00-11:00',
    '11:00-12:00': '11:00-12:00',
    '12:00-13:00': '12:00-13:00',
    '13:00-14:00': '13:00-14:00',
    '14:00-15:00': '14:00-15:00',
    '15:00-16:00': '15:00-16:00',
    '16:00-17:00': '16:00-17:00',
    '17:00-18:00': '17:00-18:00',
    '18:00-19:00': '18:00-19:00',
    '19:00-20:00': '19:00-20:00',
    '20:00-21:00': '20:00-21:00',
    '21:00-22:00': '21:00-22:00',
    '22:00-23:00': '22:00-23:00',
    '23:00-00:00': '23:00-00:00',
    'Rang': 'Rang'
}

# Afficher chaque tableau final avec le nouvel ordre des colonnes et les nouveaux noms
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    
    # Renommer les colonnes
    tableau = tableau.rename(columns=nouveaux_noms_colonnes)
    
    display(tableau)
    print("\n" + "=" * 50 + "\n")


Tableau: ligne numéro[1]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,republique,2215941.6,0.007421,3676.321623,735.264325,367.632162,5,3,5,8,...,38.42,42.22,48.75,47.65,40.31,26.59,22.13,22.07,20.89,7
1,la defense,9256802.0,0.031002,5635.386074,5635.386074,2817.693037,1,1,nan,nan,...,68.93,79.89,103.14,108.83,84.18,53.23,26.30,14.93,14.40,9
2,hotel de ville,3625864.5,0.012143,2411.033293,1205.516646,602.758323,2,1,11,nan,...,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49,13
3,palais-royal,2411299.5,0.008076,1657.541667,828.770833,414.385417,2,1,7,nan,...,40.91,55.84,70.75,67.08,42.63,27.29,21.24,20.44,20.58,40
4,basilique de saint-denis,3991395.0,0.013367,348.549592,348.549592,174.274796,1,13,nan,nan,...,34.14,36.42,39.92,33.92,25.82,14.12,8.43,6.61,4.61,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,jussieu,1444821.0,0.004839,1019.405340,509.702670,254.851335,2,7,10,nan,...,38.64,47.61,62.35,61.45,41.98,26.16,18.39,17.92,17.77,114
124,mairie d'issy,2874138.0,0.009626,979.186173,979.186173,489.593087,1,12,nan,nan,...,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20,115
125,pernety,2173567.0,0.007279,692.335762,692.335762,346.167881,1,13,nan,nan,...,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96,162
126,notre-dame-de-lorette,1893828.0,0.006343,639.111922,639.111922,319.555961,1,12,nan,nan,...,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33,188




Tableau: ligne numéro[2]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,concorde,1.133740e+06,0.009798,1252.275152,417.425051,208.712525,3,1,8,12,...,40.62,54.21,66.58,69.70,54.94,27.15,19.30,18.36,17.88,86
1,pasteur,1.513143e+06,0.013077,1109.536689,554.768344,277.384172,2,6,12,nan,...,35.89,38.08,43.50,44.12,35.49,20.42,12.83,11.10,10.56,105
2,victor hugo,2.801041e+06,0.024207,830.791135,830.791135,415.395568,1,2,nan,nan,...,39.10,43.64,48.86,48.08,43.57,22.99,12.69,12.26,12.33,123
3,alexandre dumas,2.476608e+06,0.021403,776.476449,776.476449,388.238225,1,2,nan,nan,...,36.13,39.34,43.01,39.28,33.30,22.87,14.42,12.51,10.12,137
4,abbesses,1.513884e+06,0.013083,386.886573,386.886573,193.443286,1,12,nan,nan,...,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35,233
5,trinite-d'estienne d'orves,1.051982e+06,0.009091,531.785099,531.785099,265.892549,1,12,nan,nan,...,38.18,44.55,51.19,54.76,49.49,33.64,26.85,21.44,16.85,280
6,saint-georges,7.515550e+05,0.006495,244.403647,244.403647,122.201824,1,12,nan,nan,...,37.65,38.83,47.79,42.88,34.41,21.11,16.17,26.32,17.00,291
7,assemblee nationale,6.115120e+05,0.005285,233.615898,233.615898,116.807949,1,12,nan,nan,...,35.53,47.38,65.30,64.65,41.95,25.79,16.80,15.93,18.23,296
8,saint-lazare,6.625677e+06,0.057259,18616.056763,3723.211353,1861.605676,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
10,nation,1.512699e+06,0.013073,1605.467995,401.366999,200.683499,4,1,2,6,...,73.30,78.47,90.17,88.67,79.27,49.96,33.56,27.09,22.85,20




Tableau: ligne numéro[3]


...

,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,concorde,1.133740e+06,0.018686,1252.275152,417.425051,208.712525,3,1,8,12,...,40.62,54.21,66.58,69.70,54.94,27.15,19.30,18.36,17.88,86
1,pasteur,1.513143e+06,0.024939,1109.536689,554.768344,277.384172,2,6,12,nan,...,35.89,38.08,43.50,44.12,35.49,20.42,12.83,11.10,10.56,105
2,abbesses,1.513884e+06,0.024951,386.886573,386.886573,193.443286,1,12,nan,nan,...,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35,233
3,trinite-d'estienne d'orves,1.051982e+06,0.017338,531.785099,531.785099,265.892549,1,12,nan,nan,...,38.18,44.55,51.19,54.76,49.49,33.64,26.85,21.44,16.85,280
4,saint-georges,7.515550e+05,0.012387,244.403647,244.403647,122.201824,1,12,nan,nan,...,37.65,38.83,47.79,42.88,34.41,21.11,16.17,26.32,17.00,291
5,assemblee nationale,6.115120e+05,0.010079,233.615898,233.615898,116.807949,1,12,nan,nan,...,35.53,47.38,65.30,64.65,41.95,25.79,16.80,15.93,18.23,296
6,saint-lazare,6.625677e+06,0.109201,18616.056763,3723.211353,1861.605676,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
8,marcadet-poissonniers,1.991002e+06,0.032815,1335.092784,667.546392,333.773196,2,4,12,nan,...,37.81,39.17,43.07,42.94,37.58,27.00,16.68,10.82,7.56,55
9,convention,3.734750e+06,0.061554,1207.160606,1207.160606,603.580303,1,12,nan,nan,...,38.31,38.63,37.75,38.46,33.51,20.97,13.06,9.80,7.74,67
10,pigalle,1.750916e+06,0.028858,1056.782899,528.391449,264.195725,2,2,12,nan,...,34.97,37.66,39.98,40.12,36.95,26.15,21.85,28.69,26.75,77




Tableau: ligne numéro[13]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,basilique de saint-denis,3991395.0,0.048066,348.549592,348.549592,174.274796,1,13,nan,nan,...,34.14,36.42,39.92,33.92,25.82,14.12,8.43,6.61,4.61,54
1,plaisance,3521753.0,0.042411,1134.947464,1134.947464,567.473732,1,13,nan,nan,...,34.61,37.11,39.72,38.27,32.66,22.15,12.53,8.94,7.01,76
2,malakoff-plateau de vanves,2242320.0,0.027003,773.738210,773.738210,386.869105,1,13,nan,nan,...,35.59,34.75,39.62,41.65,31.21,18.21,9.74,7.87,5.92,155
3,guy moquet,2205313.0,0.026557,894.240752,894.240752,447.120376,1,13,nan,nan,...,34.54,35.67,37.81,36.78,33.77,20.83,12.24,9.86,8.41,158
4,champs-elysees-clemenceau,954502.5,0.011495,636.167476,318.083738,159.041869,2,1,13,nan,...,34.39,47.75,62.33,67.18,52.39,31.77,25.55,24.32,23.10,187
5,les agnettes,1890356.0,0.022765,610.874772,610.874772,305.437386,1,13,nan,nan,...,32.00,33.13,34.57,31.37,25.58,17.10,10.05,7.16,5.00,189
6,la fourche,1829271.0,0.022029,550.641119,550.641119,275.320560,1,13,nan,nan,...,33.81,36.30,40.81,40.02,37.61,24.24,15.98,12.67,11.38,198
7,varenne,782697.0,0.009426,270.470695,270.470695,135.235347,1,13,nan,nan,...,39.96,55.50,69.74,68.24,34.46,17.61,11.87,10.16,16.57,290
8,saint-lazare,6625676.8,0.079790,18616.056763,3723.211353,1861.605676,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
10,chatillon-montrouge,5034012.0,0.060622,1543.318237,1543.318237,771.659118,1,13,nan,nan,...,32.62,34.96,41.77,41.56,29.17,16.87,9.27,6.52,5.61,35




Tableau: ligne numéro[14]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,saint-lazare,6625676.8,0.119400,18616.056763,3723.211353,1861.605676,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
2,mairie de saint-ouen,2415405.0,0.043528,846.124381,423.062191,211.531095,2,13,14,nan,...,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66,39
3,pont cardinet,4168538.0,0.075121,1497.119048,1497.119048,748.559524,1,14,nan,nan,...,35.56,37.63,43.22,45.32,37.99,23.12,14.08,7.68,6.32,50
4,pyramides,1981349.0,0.035706,1287.978261,643.989130,321.994565,2,7,14,nan,...,40.24,48.26,63.20,67.67,51.92,33.49,24.30,22.48,18.01,56
5,clichy saint-ouen,3420852.0,0.061647,1180.762691,1180.762691,590.381346,1,14,nan,nan,...,34.42,42.10,54.71,54.96,36.11,20.36,12.13,6.13,4.06,84
6,bibliotheque,11104474.0,0.200112,2272.627107,2272.627107,1136.313554,1,14,nan,nan,...,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76,6
7,chatelet,1670158.8,0.030098,5877.878321,1175.575664,587.787832,5,1,4,7,...,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60,10
8,olympiades,5214595.0,0.093971,1895.432969,1895.432969,947.716484,1,14,nan,nan,...,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47,31
9,gare de lyon,9546825.0,0.172042,5298.853444,1766.284481,883.142241,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
11,bercy,1942106.0,0.034998,1407.759105,703.879553,351.939776,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61


Enregistrer les tableaux

In [150]:
for k in numéro:
    k_str = str(k)
    tableaux[f'ligne numéro[{k_str}]'].to_csv(f'ligne_numero_{k_str}.csv', index=False) 